<a href="https://colab.research.google.com/github/alexis-castellanos/umsi-siads-696/blob/main/siads696_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##########################################
# PYTORCH IMPLEMENTATION
##########################################

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_STATE)

# Verify environment
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")

# Define data loading function
def load_and_prepare_data(sample_size=5000):
    """Load and preprocess UCI Adult income dataset"""
    # Column names for the dataset
    column_names = [
        'age', 'workclass', 'fnlwgt', 'education', 'education_num',
        'marital_status', 'occupation', 'relationship', 'race', 'sex',
        'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'
    ]

    # Load data
    print("Loading UCI Adult Income dataset...")
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"

    # Try different loading parameters
    try:
        data = pd.read_csv(url, names=column_names, sep=", ", engine="python", na_values=" ?")
    except:
        try:
            data = pd.read_csv(url, names=column_names, sep=",", na_values="?")
        except:
            data = pd.read_csv(url, names=column_names, sep=None, engine="python",
                             na_values="?", delim_whitespace=True)

    # Sample data if needed
    if sample_size < len(data):
        # Stratified sampling
        high_income = data[data['income'].str.contains('>50K')]
        low_income = data[data['income'].str.contains('<=50K')]

        high_count = min(int(sample_size * 0.3), len(high_income))
        low_count = min(sample_size - high_count, len(low_income))

        high_sample = high_income.sample(high_count, random_state=RANDOM_STATE)
        low_sample = low_income.sample(low_count, random_state=RANDOM_STATE)

        data = pd.concat([high_sample, low_sample])
        data = data.sample(frac=1, random_state=RANDOM_STATE).reset_index(drop=True)

    print(f"Dataset shape: {data.shape}")
    print(f"Income distribution: {data['income'].value_counts().to_dict()}")

    # Separate features and target
    X = data.drop('income', axis=1)
    y = data['income'].apply(lambda x: 1 if '>50K' in x else 0)

    # Identify column types
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

    # Create preprocessing pipelines
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ]
    )

    return X, y, preprocessor

# Define PyTorch model
class IncomeClassifier(nn.Module):
    def __init__(self, input_dim):
        super(IncomeClassifier, self).__init__()

        # Input layer
        self.layer1 = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Hidden layers
        self.layer2 = nn.Sequential(
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        self.layer4 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        self.layer5 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Output layer
        self.output = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x

# Function to train model
def train_model(X, y, preprocessor):
    """Train PyTorch model with GPU acceleration if available"""
    # Set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
    )

    # Process features
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # Get feature dimension
    input_dim = X_train_processed.shape[1]
    print(f"Input dimension after preprocessing: {input_dim}")

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_processed)
    y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test_processed)
    y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)

    # Create validation set
    val_size = int(len(X_train_tensor) * 0.2)
    X_val_tensor = X_train_tensor[-val_size:]
    y_val_tensor = y_train_tensor[-val_size:]
    X_train_tensor = X_train_tensor[:-val_size]
    y_train_tensor = y_train_tensor[:-val_size]

    # Create data loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Initialize model
    model = IncomeClassifier(input_dim=input_dim)
    model = model.to(device)

    # Print model summary
    print(model)
    print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")

    # Define loss and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, min_lr=0.00001, verbose=True
    )

    # Set training parameters
    num_epochs = 100
    patience = 10
    counter = 0
    best_val_loss = float('inf')
    best_model_state = None

    # Metrics tracking
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    # Start timer
    start_time = time.time()

    # Training loop
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for inputs, labels in train_pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Stats
            running_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update progress bar
            train_pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100 * correct/total:.2f}%'
            })

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        val_losses.append(val_loss)
        val_accs.append(val_acc)

        # Update scheduler
        scheduler.step(val_loss)

        # Print stats
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            best_model_state = model.state_dict().copy()
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Training time
    total_time = time.time() - start_time
    print(f"Training completed in {total_time:.2f} seconds")

    # Load best model
    if best_model_state:
        model.load_state_dict(best_model_state)

    # Evaluate on test set
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()

            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Convert to numpy
    all_preds = np.array(all_preds).flatten()
    all_probs = np.array(all_probs).flatten()
    all_labels = np.array(all_labels).flatten()

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)

    # Display results
    print(f"\nTest Accuracy: {accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))

    # Plot confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
                xticklabels=['<=50K', '>50K'],
                yticklabels=['<=50K', '>50K'])
    plt.title('PyTorch Neural Network Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('pytorch_confusion_matrix.png', dpi=300, bbox_inches='tight')
    plt.show()

    # Save model
    torch.save(model.state_dict(), 'income_classifier.pt')
    print("Model saved to 'income_classifier.pt'")

    return model, {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs,
        'accuracy': accuracy,
        'training_time': total_time
    }

# Main execution
if __name__ == "__main__":
    # Load and preprocess data
    X, y, preprocessor = load_and_prepare_data(sample_size=5000)

    # Train model
    model, metrics = train_model(X, y, preprocessor)

    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(metrics['train_losses'], label='Train')
    plt.plot(metrics['val_losses'], label='Validation')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(metrics['train_accs'], label='Train')
    plt.plot(metrics['val_accs'], label='Validation')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()

    plt.tight_layout()
    plt.savefig('pytorch_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Final accuracy: {metrics['accuracy']:.4f}")
    print(f"Training time: {metrics['training_time']:.2f} seconds")

    # Print hardware utilization info
    if torch.cuda.is_available():
        print(f"\nGPU Memory Usage:")
        print(f"Allocated: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
        print(f"Cached: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")